In [ ]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd

import os

import datashader as ds
import colorcet


#, pandas as pd, colorcet
#df  = pd.read_csv('census.csv')
#cvs = ds.Canvas(plot_width=850, plot_height=500)
#agg = cvs.points(df, 'longitude', 'latitude')
#img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')



In [ ]:
from astropy import units as u
from astropy.coordinates import SkyCoord

In [ ]:
#infilenames = ['GaiaSource_000000-003111.csv.gz']
#infilenames += ['GaiaSource_668805-668856.csv.gz']
#infilenames += ['GaiaSource_472714-472767.csv.gz']
#infilenames += ['GaiaSource_786097-786431.csv.gz']


path = '/home/bellis/GAIA_data'

files = (filename for filename in os.listdir(path)
         if os.path.isfile(os.path.join(path,filename)) and filename.find('.h5')>=0)

infilenames = []
for f in files:
    infilenames.append(f"{path}/{f}")
    #print(f)
          
print(f"# files: {len(infilenames)}")

In [ ]:
li = []

for infilename in infilenames:
    print(infilename)
    df = None
    if infilename.find('.csv.gz')>=0:
        df = pd.read_csv(infilename, compression='gzip')
    elif infilename.find('.h5')>=0:
        df = pd.read_hdf(infilename)

    li.append(df)
    
df = pd.concat(li, axis=0, ignore_index=True)
    

Should also check out VisIt

https://visit-dav.github.io/visit-website/about/

And Paraview

https://www.paraview.org/


In [ ]:
#df = pd.read_hdf(infilenames[0])
cvs = ds.Canvas(plot_width=850, plot_height=500)
agg = cvs.points(df, 'ra', 'dec')
img = ds.tf.shade(agg, cmap=colorcet.fire, how='log')

img

# Write out small sample



In [ ]:
nvals = len(df)
print(nvals)

nsamples = 100

x = np.arange(0,nvals,1).astype(int)
np.random.shuffle(x)

indices = x[0:nsamples]

print(indices)

df_sample = df.iloc[indices]

name = f"gaia_subset_n{nsamples}.h5"

df_sample.to_hdf(name,'df')

df_sample

plt.figure()
plt.plot(df_sample['ra'],df_sample['dec'],'.',markersize=1)

In [ ]:
len(df_sample)

[GAIA source documentation](https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html)

In [ ]:
df.columns

In [ ]:
np.unique(df['dr2_radial_velocity'])

In [ ]:
rv = df['dr2_radial_velocity']
rv = rv.dropna()

print(f"{len(rv)} entries with rv as a number")
print(f"{len(df)} total entries")

In [ ]:
plt.hist(rv,bins=200);
plt.yscale('log')

In [ ]:
#dfgood = df[~df['dr2_radial_velocity'].isnull()]
dfgood = df

In [ ]:
len(dfgood)

dfgood['ra']

In [ ]:
x = dfgood['dr2_radial_velocity']
y = dfgood['phot_g_mean_mag']
y0 = df['phot_g_mean_mag']

plt.figure(figsize=(12,8))
plt.hist(y0,bins=50,range=(0,25))
plt.hist(y,bins=50,range=(0,25));

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x,y,'.',alpha=0.5,markersize=0.5)

https://en.wikipedia.org/wiki/Galactic_coordinate_system

https://astronomy.stackexchange.com/questions/35536/transforming-galactic-coordinates-in-cartesian-with-distance

https://en.wikipedia.org/wiki/Stellar_parallax

[Parsec](https://en.wikipedia.org/wiki/Parsec) is the units for $1/\textrm{parallax}$. 1 parsec = 3.26 light-years.

I think the units of parsecs is milliarcsecs (mas).

What to do about negative parallax?

https://astronomy.stackexchange.com/questions/26250/what-is-the-proper-interpretation-of-a-negative-parallax

Also, ecliptic coordinates are in the data

https://en.wikipedia.org/wiki/Ecliptic_coordinate_system

and l and b are in there explicitly. 

In [ ]:
c_icrs = SkyCoord(ra=10.68458*u.degree, dec=41.26917*u.degree, frame='icrs')
c_icrs.galactic

In [ ]:
#def galactic_coordinates(ra, dec, parallax):
def galactic_coordinates(l, b, parallax):

    #c = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    
    #l = c.galactic.l.rad
    #b = c.galactic.b.rad
    
    lrad = np.deg2rad(l)
    brad = np.deg2rad(b)
    
    # Assume that parallax is passed in as milliarcsec
    conversion = 0.001 # Convert milliarcsec to arcsec
    one_over_p = 1/(parallax*conversion)

    x = one_over_p*np.cos(brad)*np.cos(lrad)
    y = one_over_p*np.cos(brad)*np.sin(lrad)
    z = one_over_p*np.sin(brad)
    
    return x,y,z
    

In [ ]:
ra_org = dfgood['ra']
dec_org = dfgood['dec']
p_org = dfgood['parallax']

l_org = dfgood['l']
b_org = dfgood['b']


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(ra_org,bins=50)

plt.subplot(1,3,2)
plt.hist(dec_org,bins=50)

plt.subplot(1,3,3)
plt.hist(p_org,bins=50)

plt.figure(figsize=(12,12))
plt.plot(ra_org,dec_org,'.',markersize=0.5,alpha=0.2)

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.hist(p_org,bins=50)


plt.subplot(2,2,2)
plt.hist(p_org[(p_org<50) & (p_org>0)],bins=50)

plt.subplot(2,2,3)
plt.hist(p_org[p_org>50],bins=50)

plt.subplot(2,2,4)
plt.hist(p_org[p_org<0.1],bins=50)

plt.figure()
plt.hist(1/p_org,bins=100)

1

In [ ]:
x,y,z = galactic_coordinates(10,20,0.5)

print(x,y,z)

Someone else's visualization of within 3000 parsecs

https://www.cosmos.esa.int/web/gaia/iow_20180614

In [ ]:
#mask = (p_org>0.1) & (ra_org == ra_org)
mask = (p_org>0.1)


#ra = ra_org[mask].values
#dec = dec_org[mask].values

l = l_org[mask].values
b = b_org[mask].values

p = p_org[mask].values
            

#x,y,z = galactic_coordinates(ra, dec, p)
x,y,z = galactic_coordinates(l, b, p)


#fig = plt.figure(figsize=(8,8))
#ax = fig.add_subplot(projection='3d')
#
#plt.plot(x,y,z,'.',markersize=0.5,alpha=0.2)

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,3,1)
plt.hist(x,bins=50)

plt.subplot(1,3,2)
plt.hist(y,bins=50)

plt.subplot(1,3,3)
plt.hist(z,bins=50)

1

In [ ]:
rv = dfgood['dr2_radial_velocity']

In [ ]:
len(rv)

In [ ]:
plt.figure()
plt.hist(rv,bins=100)
plt.yscale('log')

1

In [ ]:
x,y,z

In [ ]:
rv

In [ ]:
df.columns

In [ ]:
df['ecl_lon']

In [ ]:
dfgood['parallax']

In [ ]:
#!pip install plotly

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


In [ ]:
#px.scatter_3d(x,y,z,'.')
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(size=1))])

fig.show()

Proper motion

https://gea.esac.esa.int/archive/documentation/GEDR3/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html
    

https://en.wikipedia.org/wiki/Proper_motion#:~:text=Proper%20motion%20is%20the%20astrometric,of%20the%20more%20distant%20stars.
    
 
pmra,pmdec
    

In [ ]:
pmra = dfgood['pmra']
pmdec = dfgood['pmdec']

In [ ]:
#mask = pmdec<1600
mask = pmdec<1000


#plt.plot(pmra[mask],pmdec[mask],'.')

In [ ]:
p = dfgood['parallax']
pm = dfgood['pm']


conversion = 0.001 # Convert milliarcsec to arcsec
one_over_p = 1/(p*conversion)
distance = one_over_p

vmag = np.sqrt(rv[mask]**2 + (distance[mask]*(pm[mask]/3.1e8))**2)

In [ ]:
plt.hist(vmag,bins=100,range=(0,1000));

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)

plt.hist(rv[mask],bins=100,range=(0,1000));

plt.subplot(1,2,2)

plt.hist(distance[mask]*pm[mask]/3.1e8,bins=100,range=(0,0.001))

1;

In [ ]:
plt.hist(pm[mask]/3.1e8,bins=100);

In [ ]:
max(distance[mask]*pm[mask]/3.1e8)

In [ ]:
nvals = len(df)
print(nvals)

nsamples = 100000

x = np.arange(0,nvals,1).astype(int)
np.random.shuffle(x)

indices = x[0:nsamples]

#print(indices)

df_sample = df.iloc[indices]

name = f"gaia_subset_n{nsamples}.h5"

df_sample.to_hdf(name,'df')

df_sample

p = df_sample['parallax']

#mask = (p_org>0.1) & (ra_org == ra_org)
mask = (p>0.1)


#ra = ra_org[mask].values
#dec = dec_org[mask].values

l = df_sample[mask]['l'].values
b = df_sample[mask]['b'].values

p = p[mask].values
            

#x,y,z = galactic_coordinates(ra, dec, p)
x,y,z = galactic_coordinates(l, b, p)

In [ ]:
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,
                                   mode='markers', marker=dict(size=1))])

fig.show()